# Cartpole

### Steps

For Deep Q Agent, define:
- the DNN model
- action selection procedure
  - will need epsilon and epsilon decay parameters
- memory and replay functions for learning
  - will need learning rate and batch size parameters
  - minimize cost (MSE) where target value is current reward + discounted (by factor of gamma) predicted reward of next state

Uses https://keon.io/deep-q-learning/ as reference


In [1]:
import gym
import matplotlib.pyplot as plt
import numpy as np
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
class DeepQAgent(object):
  
  
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size

        self.learning_rate = 0.001
        self.gamma = 0.1

        self.epsilon = 0.01 # set low if using pre-trained weights
        self.epsilon_decay = 0.005
        self.min_epsilon = 0.01

        self.replay_buffer = deque(maxlen=10000) # from stackoverflow: https://stackoverflow.com/questions/23487307/python-deque-vs-list-performance-comparison
        self.model = self.build_model()

    
    def build_model(self):
    
        # Construct a NN with two hidden layers, rectified linear unit activation
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu')) # input layer size (state_size,)
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear')) # output layer size (action_size,). Linear because we want to map to Q values, e.g. not softmax probabilities

        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))

        return model

  
    def choose_action(self, state):
        if (np.random.rand() <= self.epsilon): # explore with probability of epsilon
            return env.action_space.sample()
        else:
            q_values = self.model.predict(state)
            return np.argmax(q_values[0]) # choose the action that maximizes q value

    
    def add_to_replay_buffer(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

    
    def replay(self, batch_size):
        # sample a batch from memory
        batch = random.sample(self.replay_buffer, batch_size)

        # for the sampled data in batch
        for state, action, reward, next_state, done in batch:
            target = reward

            if not done: # if the state isnt terminal, predict the future discounted reward.  Else, target value is equal to the reward received
                target = reward + (1 - self.gamma) * np.amax(self.model.predict(next_state)[0]) # return the highest VALUE (not the action), since we are updating target reward

            # train the agent to map current state to future (discounted) reward
            q_values = self.model.predict(state) # predict the values (since we don't know what both actions would have yielded)
            q_values[0][action] = target # and then reassign the taken action value to the reward / target

            self.model.fit(state, q_values, epochs=1, verbose=0)

        if self.epsilon > self.min_epsilon:
            self.epsilon *= (1 - self.epsilon_decay)

      
    def load_weights(self, name):
        self.model.load_weights(name)

    
    def save_weights(self, name):
        self.model.save_weights(name)
        

In [3]:
# helper function to transpose returned state so that it can serve as input to model

def transpose(state, dimension):
    return np.reshape(state, [1, dimension])

In [4]:
env = gym.make("CartPole-v0")
observation = env.reset()

# print(vars(env.action_space))
# print(vars(env.observation_space))

action_size = env.action_space.n
state_size = env.observation_space.shape[0]

[2018-02-18 18:16:54,489] Making new env: CartPole-v0


In [8]:
EPISODES = 100
MAX_STEPS = 500 # because we don't want this to go on forever (should the model ever get that good ;) )
SOLVED_STEPS = 195 # openAI says that solving is averaging at least 195 steps per trial
BATCH_SIZE = 32

FILE_NAME = 'saved_models/cartpole_dqn_weights.h5'


agent = DeepQAgent(state_size, action_size)
agent.load_weights(FILE_NAME)

timesteps = 0
max_timesteps_so_far = 0

for i in range(EPISODES):
  
    state = env.reset()
    state = transpose(state, state_size)

    for t in range(MAX_STEPS):
        # env.render() # can use this to watch visualization on a VM that supports it

        action = agent.choose_action(state)
        next_state, reward, done, _ = env.step(action)
        next_state = transpose(next_state, state_size)
        agent.add_to_replay_buffer(state, action, reward, next_state, done)

        state = next_state

        if done:
            # calculate average timesteps over episode_range number of episodes
            timesteps += t
            episode_range = 10

            if i != 0 and i % episode_range == 0:
                print("Episodes {} through {} finished after an average of {} timesteps".format(i - episode_range, i, timesteps / episode_range))
                if timesteps / episode_range > max_timesteps_so_far:
                    max_timesteps_so_far = timesteps / episode_range
        #           agent.save_weights(FILE_NAME)
        #           print('Model weights saved.')

                if timesteps / episode_range >= SOLVED_STEPS:
                    print('Episodes were a sucess!')

                timesteps = 0
            break

    if len(agent.replay_buffer) > BATCH_SIZE: # start learning once we have seen enough examples to form a batch to train on
        agent.replay(BATCH_SIZE)

# env.render()
env.reset()
env.close()

Episodes 0 through 10 finished after an average of 211.2 timesteps
Episodes were a sucess!
Episodes 10 through 20 finished after an average of 194.5 timesteps
Episodes 20 through 30 finished after an average of 193.1 timesteps
Episodes 30 through 40 finished after an average of 194.2 timesteps
Episodes 40 through 50 finished after an average of 189.5 timesteps
Episodes 50 through 60 finished after an average of 194.1 timesteps
Episodes 60 through 70 finished after an average of 197.5 timesteps
Episodes were a sucess!
Episodes 70 through 80 finished after an average of 196.8 timesteps
Episodes were a sucess!
Episodes 80 through 90 finished after an average of 192.3 timesteps
